# Ejecutar script en Azure ML

## Conectar a workspace

In [ ]:
! pip show azure-ai-ml

In [ ]:
# conectar al workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Usar el SDK de Python para entrenar un modelo

In [ ]:
%%writefile src/diabetes-training.py

# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))


Overwriting /home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src/diabetes-training.py


## Generar el job

In [ ]:
from azure.ai.ml import command, MLClient
from azure.ai.ml.entities import Environment

# Definir un comando para el experimento
job = command(
    code="./src",  # Carpeta con el código
    command="python diabetes-training.py",
    environment="azureml://registries/azureml/environments/sklearn-1.1/versions/4",
    compute="ricardoenm",
    display_name="diabetes-training-ric"
)

# Enviar el experimento
returned_job = ml_client.jobs.create_or_update(job)
print(f"Experimento enviado: {returned_job}")



Uploading src (0.52 MBs): 100%|██████████| 518911/518911 [00:00<00:00, 4056938.61it/s]




Experimento enviado: type: command
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.icy_endive_pwz49yv043
environment: azureml://registries/azureml/environments/sklearn-1.1/versions/4
resources:
  instance_count: 1
  shm_size: 2g
component:
  name: icy_endive_pwz49yv043
  display_name: diabetes-training-ric
  type: command
  outputs:
    default:
      type: uri_folder
      mode: rw_mount
  command: python diabetes-training.py
  environment: azureml://registries/azureml/environments/sklearn-1.1/versions/4
  code: azureml:/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/codes/59f44e09-9bfc-449e-9718-4d6a8eb21205/versions/1
  creation_context:
    created_at: '2025-05-08T08:37:30.340287+00:00'
    created_by: Ricardo Ahumada
    created_by_type: User
  is_deterministic: true
compute: azureml:ricardoenm
properties: